In [9]:
from pyspark.sql import SparkSession
import os

# Inicializa la SparkSession
spark = SparkSession.builder \
    .appName("refine_gkg_counts") \
    .getOrCreate()

# Especifica el directorio local donde están los archivos CSV
base_dir = '/home/oscar/budasbi-repos/factored-datathon-2024-voyager/bucket_contents/raw/gkg_counts'

# Recorrer el directorio y sus subdirectorios para obtener todos los archivos CSV
gkg_files = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".csv"):
            gkg_files.append(os.path.join(root, file))


24/08/19 00:11:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/19 00:11:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [10]:
spark.sparkContext

<SparkContext master=local[*] appName=refine_gkg_counts>

In [11]:

# Si tienes varios archivos, puedes concatenarlos para leerlos todos juntos
if gkg_files:
    gkg_counts = spark.read.csv(gkg_files, header=True, sep='\t',inferSchema=True)
    print()
    gkg_counts.show()  # Muestra las primeras filas del DataFrame
else:
    print("No se encontraron archivos CSV en el directorio especificado.")



+--------+-------+--------------------+------+-------------------+--------+--------------------+---------------+------------+-------+--------+-------------+--------------------+--------------------+--------------------+
|    DATE|NUMARTS|           COUNTTYPE|NUMBER|         OBJECTTYPE|GEO_TYPE|        GEO_FULLNAME|GEO_COUNTRYCODE|GEO_ADM1CODE|GEO_LAT|GEO_LONG|GEO_FEATUREID|       CAMEOEVENTIDS|             SOURCES|          SOURCEURLS|
+--------+-------+--------------------+------+-------------------+--------+--------------------+---------------+------------+-------+--------+-------------+--------------------+--------------------+--------------------+
|20240805|      1|                KILL|    10|               null|       1|             Germany|             GM|          GM|   51.5|    10.5|           GM|1191056014,119105...|           yahoo.com|https://news.yaho...|
|20240805|      2|CRISISLEX_C03_WEL...|     2|               null|       4|Pokrovsk, Mordovi...|             RS|       

In [12]:
gkg_counts.count()#1441421

307576

In [13]:
gkg_counts.printSchema()

root
 |-- DATE: integer (nullable = true)
 |-- NUMARTS: integer (nullable = true)
 |-- COUNTTYPE: string (nullable = true)
 |-- NUMBER: long (nullable = true)
 |-- OBJECTTYPE: string (nullable = true)
 |-- GEO_TYPE: integer (nullable = true)
 |-- GEO_FULLNAME: string (nullable = true)
 |-- GEO_COUNTRYCODE: string (nullable = true)
 |-- GEO_ADM1CODE: string (nullable = true)
 |-- GEO_LAT: double (nullable = true)
 |-- GEO_LONG: double (nullable = true)
 |-- GEO_FEATUREID: string (nullable = true)
 |-- CAMEOEVENTIDS: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- SOURCEURLS: string (nullable = true)



In [ ]:
gkg_colums=['date','numarts', 'count_type', 'number', 'object_type', 'geo_type', 'geo_fullname', 'geo_country_code', 'geo_adm1_code', 'geo_lat', 'geo_long','geo_feature_id', 'cameo_event_ids', 'sources', 'source_urls' ]
#publication_date -> to date
#geo_type -> replace nulls with 0
#geo_type = {1:'Country', 2:'US State', 3:'US City', 4: 'World City', 5: 'World State', 0: 'Not Assigned'}
#geo_country_code resolve with 
#geo_adm1_code -> https://github.com/mysociety/gaze/blob/master/data/us-states.csv
#Explote cameo_event_ids

In [ ]:
df.show(truncate=False)

In [ ]:
df.printSchema()

In [ ]:
new_column_names = ['Date', ]

In [ ]:
df.select('DATE', 'NUMARTS').show()

In [ ]:
df.createOrReplaceTempView('gkgcounts')
limit10 = spark.sql("select * from gkgcounts limit 10")
limit10.show(truncate=False)

In [ ]:
from pyspark.sql.functions import split, explode
df_split = df.withColumn("CAMEOEVENTIDS_split", split(df["CAMEOEVENTIDS"], ","))
df_split.show(truncate=False)

In [ ]:
df_exploded = df_split.withColumn("CAMEOEVENTIDS", explode(df_split["CAMEOEVENTIDS_split"]))
df_exploded.show(truncate=False)

In [ ]:
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


In [8]:

# Detener la sesión de Spark al finalizar
spark.stop()


In [ ]:
sc